# Set Bert

In [24]:
from transformers import AutoTokenizer, DistilBertModel
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-multilingual-cased", cache_dir='./cache/tokenizer')
distilBertModel = DistilBertModel.from_pretrained("distilbert/distilbert-base-multilingual-cased", cache_dir='./cache/model')

In [2]:
# get some vocab info
import random
random_tokens = random.sample(list(tokenizer.vocab), 10)
random_ids = [tokenizer.vocab[token] for token in random_tokens]

print("{0:20}{1:15}".format("token", "id"))
print("-" * 25)
for t, i in zip(random_tokens, random_ids):
    print("{0:15}{1:10}".format(t, i))

token               id             
-------------------------
##etem             103379
тăршшĕ              36571
卷                    2700
mor                 24984
üle                 29545
缶                    6389
رنگ                 42040
##֊                111148
×                     247
##cilla             92952


In [25]:
distilBertModel.eval()

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [26]:
input = tokenizer(["嗨", "hello"], return_tensors="pt", padding=True, truncation=True)    # Not support batch str input (because of torch.Tensor)
seqTensor = distilBertModel(**input, output_hidden_states=True)

In [5]:
print(seqTensor.hidden_states[0].shape)

torch.Size([2, 4, 768])


In [6]:
print(len(seqTensor.hidden_states))

7


In [7]:
print(input)

{'input_ids': tensor([[  101,   100,   102,     0],
        [  101, 61694, 10133,   102]]), 'attention_mask': tensor([[1, 1, 1, 0],
        [1, 1, 1, 1]])}


# Set BertDecoder

In [18]:
from utils.DevConf import DevConf
devConf = DevConf('cpu')

In [19]:
from utils.AttnBlocksConf import AttnBlocksConf

In [20]:
from model.BertDecoder.SentiClassifier import SentiClassifier
from utils.const import BlockType

In [21]:
mapper = SentiClassifier(6, AttnBlocksConf(768, 12, nKVHead=6), BlockType.LAST, devConf=devConf)

In [22]:
import torch

In [28]:
ans = mapper.forward(seqTensor)
# print(ans.shape)
print(ans)

AttributeError: 'BaseModelOutput' object has no attribute 'to'

In [13]:
print(ans.shape)

torch.Size([2, 768])


# Combination Model

In [29]:
from model.CombinationModel import CombinationModel

In [30]:
from torch import nn

In [31]:
cModel = CombinationModel(
    nClass=5,
    distilBert=distilBertModel,
    decoder=mapper,
)

In [ ]:
cModel.forward(**tokenizer("嗨", return_tensors="pt"), returnAttnWeight=True)

(tensor([[0.0764, 0.3002, 0.1742, 0.1757, 0.2736]], grad_fn=<SoftmaxBackward0>),
 tensor([[[0.3497, 0.3305, 0.3199]]], grad_fn=<MeanBackward1>))

# Prepare Data

In [ ]:
from torchtext.datasets import IMDB
from torchtext.data.functional import to_map_style_dataset

In [ ]:
trainData = iter(IMDB(root="./data", split='train'))

In [ ]:
print(next(trainData)[1])

I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, eve

In [ ]:
from torch import Tensor
import torch

In [ ]:
def collate_fn(batch)->tuple[Tensor, Tensor, Tensor]:
    target =[]
    datas = []

    for (label, data) in batch:
        datas.append(data)
        target.append(0 if label == 'neg' else 1)

    encoding = tokenizer(datas, return_tensors='pt', padding=True, truncation=True, max_length=512)
    ids = encoding['input_ids']
    mask = encoding['attention_mask']
    
    return ids, mask, torch.tensor(target, dtype=torch.int16)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
trainLoader = DataLoader(to_map_style_dataset(trainData), collate_fn=collate_fn, batch_size=16, shuffle=True)

In [ ]:
next(iter(trainLoader))

(tensor([[  101, 10747, 18379,  ...,     0,     0,     0],
         [  101, 17443, 49307,  ..., 10833, 10124,   102],
         [  101, 76783, 10230,  ...,     0,     0,     0],
         ...,
         [  101,   146, 10134,  ...,     0,     0,     0],
         [  101, 14027, 18379,  ...,     0,     0,     0],
         [  101, 12689,   169,  ...,   118, 18923,   102]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1]]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int16))

In [ ]:
import torch

In [ ]:
import torch
from einops import repeat

In [ ]:
x = torch.ones(2, requires_grad=True)
y = repeat(x, "l -> b l", b=2) * 2
# with torch.no_grad():
#     for i in range(3):
#         y = 2*y
#     # x.retain_grad()
y1 = 2*y[0]
y2 = y[1]
z = y1.sum() + y2.sum()

In [ ]:
print(x)
print(y)

tensor([1., 1.], requires_grad=True)
tensor([[4., 4.],
        [4., 4.]], grad_fn=<MulBackward0>)


In [ ]:
z.backward()

In [ ]:
print(x.grad)

tensor([6., 6.])


In [ ]:
strlist = ["a", "b", "c", "d", "e"]
concatStr = ""
for i in strlist:
    concatStr+=i
    concatStr+=" "
print(concatStr)

a b c d e 


## Loss function

In [ ]:
from torch import nn

In [ ]:
loss_fn = nn.BCELoss()

In [ ]:
import torch

In [ ]:
loss_fn(torch.tensor([[0.1, 0.9, 0.2], [0.1, 0.9, 0.5]]), torch.tensor([[1, 2], [1]])) 

ValueError: expected sequence of length 2 at dim 1 (got 1)

In [1]:
import torch
from torch import nn

In [ ]:
x = nn.Linear(1, 5, device='cuda')
layer = nn.Linear(5, 5, device='cuda')
y = layer(x(torch.tensor([1.0], device='cuda')))
z = y.sum()
z.backward()
print(x.weight.grad)

tensor([[-0.2207],
        [-0.7934],
        [ 0.1053],
        [ 0.6633],
        [-0.9146]], device='cuda:0')


In [2]:
x = torch.randn(6)
print(x.shape)
x = x.view(-1, 2)
print(x.shape)

torch.Size([6])
torch.Size([3, 2])


In [5]:
x.to('mps')
x[0]

tensor([0.2045, 0.1568])

In [25]:
x = torch.tensor([[1, 0], [0, 0], [1, 1]])
x = x.repeat(1, 2)
print(x.shape)
x = x.view(x.shape[0], -1, 2)
print(x.shape)

torch.Size([3, 4])
torch.Size([3, 2, 2])


In [26]:
print(x)

tensor([[[1, 0],
         [1, 0]],

        [[0, 0],
         [0, 0]],

        [[1, 1],
         [1, 1]]])


In [27]:
print(x[:, 0, :])

tensor([[1, 0],
        [0, 0],
        [1, 1]])


In [30]:
x[:, 1] = 1 - x[:, 1]

In [31]:
print(x)

tensor([[[1, 0],
         [1, 0]],

        [[0, 0],
         [0, 0]],

        [[1, 1],
         [1, 1]]])


In [41]:
x = torch.tensor([1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0]).view(1, 9, 2)
print(torch.argmax(x, dim=2).squeeze())

tensor([0, 1, 0, 0, 0, 1, 1, 0, 0])
